In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import re
import pandas as pd
import os


url = os.chdir(r'gdrive/MyDrive/data')
os.getcwd()

In [ ]:
files = os.listdir(url)
files

In [ ]:
k = ['train_da', 'train_bg', 'train_cs', 'train_de', 'train_el', 'train_es', 'train_et', 'train_fi',
     'train_fr', 'train_hu', 'train_it', 'train_lt', 'train_lv', 'train_nl', 'train_pl', 'train_pt',
     'train_ro', 'train_sk', 'train_sl', 'train_sv', 'test', 'test_submission']
     
lang = {'train_da':[], 'train_bg':[], 'train_cs':[], 'train_de':[], 'train_el':[], 'train_es':[], 'train_et':[],
        'train_fi':[], 'train_fr':[], 'train_hu':[], 'train_it':[], 'train_lt':[], 'train_lv':[], 'train_nl':[],
        'train_pl':[], 'train_pt':[], 'train_ro':[], 'train_sk':[], 'train_sl':[], 'train_sv':[], 'test':[], 'test_submission':[]}

i = 0
for file in files:
    with open(file, "r", encoding='utf-8') as f:
        for line in f.readlines():
            lang[k[i]].append(line)
    i += 1

In [ ]:
# re - библиотека для работы с регулярными выражениями и строками
for l in lang:
    if re.findall('train', l):
        k = l.split('train_')[1]
        print(l, '|', k)

In [ ]:
# создание тренировочного набора
train = pd.DataFrame(columns=['text', 'language'])

for l in lang:
    if re.findall('train', l):
        tmp = pd.DataFrame()
        tmp['text'] = lang[l]
        tmp['language'] = l.split('train_')[1]
        train = pd.concat([train, tmp], ignore_index=True)

print(train)

In [ ]:
print(f'What languages are represented in the data: {train["language"].unique()}')
print('-'*100)
print(f'How many sentences are presented for each language: \n{train["language"].value_counts()}')

In [ ]:
test = pd.DataFrame(columns=['text'])
tmp = []

for word in lang['test']:
    tmp.append(word.split(',')[1].split('\n')[0])

test['text'] = tmp
test

In [ ]:
word = []
lang = []
for i in range(300000):
  p = '''!()-[]{};?@#$%:'"\,./^&;*_0123456789'''
  s = (train['text'][i])
  new = ''
  for e in s:
    if e not in p:
      new += e  
  k = new.split()
  for j in range(len(k)):
    word.append(k[j])
    lang.append(train['language'][i])

train_2 = pd.DataFrame({'word': word, 'lang': lang})
train_2

In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 8.2 MB 8.3 MB/s 


In [ ]:
import pymorphy2

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

lm_txt = []
for i in range(len(train)):
  lm_wd = []
  words = train.loc[i]['Question'].split(' ')
  for i in words:
      lm = morph.parse(i)
      lm_wd.append(lm[0].normal_form)
  text = ' '.join(lm_wd)
  lm_txt.append(text)

train['Question'] = lm_txt

train

In [ ]:
lm_txt = []
for i in range(len(test)):
  lm_wd = []
  words = test.loc[i]['Question'].split(' ')
  for i in words:
      lm = morph.parse(i)
      lm_wd.append(lm[0].normal_form)
  text = ' '.join(lm_wd)
  lm_txt.append(text)

test

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


count_vectorizer = CountVectorizer()

X_train, X_test, y_train, y_test = train_test_split(train_2['word'], train_2['lang'], test_size=0.2, random_state=42)

X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)
test_counts = count_vectorizer.transform(test["text"])


clf = LogisticRegression(random_state=42)
clf.fit(X_train_counts, y_train)

y_predicted_counts = clf.predict(X_test_counts)

y_predicted_test = clf.predict(test_counts)
y_test_pred = pd.DataFrame(y_predicted_test, columns=['text'])
y_test_pred.reset_index(inplace=True)

y_test_pred

In [ ]:
y_test_pred.to_csv('submition_test_Rasyuk.csv', index=False)
files.download("submition_test_Rasyuk.csv")